In [2]:
from tensorflow import keras
from sklearn.metrics import confusion_matrix, ConfusionMatrixDisplay
import matplotlib.pyplot as plt
import numpy as np
import pickle


# f2 = open('data/split_0.80_0.10_0.10_aug_to_5000_down_sample_True_instance1/val_data', 'rb')
# f2 = open('data/split_0.80_0.10_0.10_aug_to_5000_down_sample_True_instance1/val_data', 'rb')
# X_sequence_val, X_feature_val, X_image_val, Y_val = pickle.load(f2)
# f2.close()
# f3 = open('data/split_0.80_0.10_0.10_aug_to_5000_down_sample_True_instance1/test_data', 'rb')
f3 = open('data/split_image_0.60_0.30_0.10_aug_to_1875_down_sample_True_instance0-9/test_data', 'rb')
X_sequence_test, X_feature_test, X_image_test, Y_test = pickle.load(f3)
# X_sequence_test, X_feature_test, Y_test = pickle.load(f3)
f3.close()
from keras_preprocessing.sequence import pad_sequences
X_sequence_test_paded = pad_sequences(X_sequence_test, dtype='float', padding='post', value = np.nan)
# X_sequence_val_paded = pad_sequences(X_sequence_val, dtype='float', padding='post', value = np.nan)
from util import preprocess
X_sequence_test_trimed = preprocess(X_sequence_test_paded)
# X_sequence_val_trimed = preprocess(X_sequence_val_paded)

def build_channel(data):
    X = []
    for image in data:
        X.append(image.reshape(128, 128, 1))
    return X
X_image_test = np.array(build_channel(X_image_test))
# X_image_val = np.array(build_channel(X_image_val))
X_feature_test = np.array(X_feature_test)
# X_feature_val = np.array(X_feature_val)
Y_test = np.array(Y_test)-1
# Y_val = np.array(Y_val)-1

In [6]:
models = []
best_list_0=[18,8,10,13,10,16,8,16,12,11]
best_list_1=[17,25,16,10,4,25,22,13,21,10]
best_list_2=[11,10,11,7,14,10,16,9,21,11]
best_list_3=[10,26,13,16,15,16,13,12,11,21]
best_acc_list_before_image = [6,5,4,3,5,8,5,4,7,8]
min_loss_list_before_image = [6,6,4,8,7,8,5,6,5,8]
best_acc_list_after_image = [34,37,9,33,11,39,31,22,6,18]
min_loss_list_after_image = [20,21,26,33,40,40,28,13,27,35]
for i in range(0,10):
    # lodaed_model = keras.models.load_model('models/bagging_before_image_0.60_0.30_0.10/rnn_with_feature%d/model_4'%i)
    lodaed_model = keras.models.load_model('models/bagging_before_image_0.60_0.30_0.10/rnn_with_feature%d/model_%d'%(i,best_acc_list_before_image[i]),compile=False)
    models.append(lodaed_model)

2022-11-15 10:15:59.404506: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 14 outputs but the _output_shapes attribute specifies shapes for 48 outputs. Output shapes may be inaccurate.
2022-11-15 10:16:00.784285: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 14 outputs but the _output_shapes attribute specifies shapes for 48 outputs. Output shapes may be inaccurate.
2022-11-15 10:16:00.794413: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 4 outputs but the _output_shapes attribute specifies shapes for 48 outputs. Output shapes may be inaccurate.
2022-11-15 10:16:01.004097: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond/while' has 14 outputs but the _output_shapes attribute specifies shapes for 48 outputs. Output shapes may be inaccurate.
2022-11-15 10:16:01.013213: W tensorflow/core/common_runtime/graph_constructor.cc:805] Node 'cond' has 4 outputs but the _output_sh

In [5]:

# def generate_batch(X_sequence, X_image,  X_feature, Y, batch_size):
# def generate_batch(X_sequence,  X_feature, Y, batch_size):
#     idx = 0
#     batch_num = len(Y)/batch_size
#     while idx < batch_num:
#         batch_x_sequence = X_sequence[idx * batch_size : (idx + 1) * batch_size]
#         batch_x_feature = X_feature[idx * batch_size : (idx + 1) * batch_size]
#         batch_y = Y[idx * batch_size : (idx + 1) * batch_size]
#         yield batch_x_sequence,  batch_x_feature, batch_y
#         idx += 1
from util import generate_batch
# test_gen = generate_batch(X_sequence_test_trimed, X_image_test, X_feature_test, Y_test, 128)      
test_gen = generate_batch(X_sequence_test_trimed, X_image_test, X_feature_test, Y_test, 128)      

y_label = Y_test

# val_gen = generate_batch(X_sequence_val_trimed, X_image_val, X_feature_val, Y_val, 128)      
# y_label = Y_val


y_predict = np.array([])
for x_sequence, x_image, x_feature, y in test_gen:
# for x_sequence,  x_feature, y in test_gen:
    results = []
    for model in models:
    # for i in (9,):
    #     model = models[i]
        results.append(model.predict({'sequence':x_sequence, 
                                        'image':x_image,
                                        'feature':x_feature}))
    average_result = np.sum(results, axis=0)
    y_predict = np.concatenate([y_predict, np.argmax(average_result, axis=-1)])
y_label = y_label.astype(np.int32) 
y_predict = y_predict.astype(np.int32) 
cm = confusion_matrix(y_true=y_label, y_pred=y_predict, normalize='true')
print(cm.shape)
# disp = ConfusionMatrixDisplay(confusion_matrix=cm)
# disp.plot(values_format='.2g')

classes = ['RRab', 'RRc','RRd', 'Blazhko','Ecl', 'EA' , 'Rot', 'LPV','δ-Scuti','ACEP', 'Cep-II']

from util import cm_plot

plotting = cm_plot(cm, classes_types=classes, classifier_model='mix Classifier', nClasses=11,
                    cmap=plt.cm.Blues)

/home/zhkang/softwares/anaconda3/envs/DL/lib/python3.9/site-packages/keras/engine/functional.py:637: UserWarning: Input dict contained keys ['image'] which did not match any model input. They will be ignored by the model.
  inputs = self._flatten_to_reference_inputs(inputs)
2022-11-15 10:15:00.625935: E tensorflow/stream_executor/cuda/cuda_blas.cc:218] failed to create cublas handle: cublas error
2022-11-15 10:15:00.625979: E tensorflow/stream_executor/cuda/cuda_blas.cc:220] Failure to initialize cublas may be due to OOM (cublas needs some free memory when you initialize it, and your deep-learning framework may have preallocated more than its fair share), or may be because this binary was not built with support for the GPU in your machine.
2022-11-15 10:15:00.625995: W tensorflow/core/framework/op_kernel.cc:1780] OP_REQUIRES failed at matmul_op_impl.h:620 : INTERNAL: Attempting to perform BLAS operation using StreamExecutor without BLAS support


InternalError: Graph execution error:

Detected at node 'model_1/feature_dense_1/MatMul' defined at (most recent call last):
    File "/home/zhkang/softwares/anaconda3/envs/DL/lib/python3.9/runpy.py", line 197, in _run_module_as_main
      return _run_code(code, main_globals, None,
    File "/home/zhkang/softwares/anaconda3/envs/DL/lib/python3.9/runpy.py", line 87, in _run_code
      exec(code, run_globals)
    File "/home/zhkang/softwares/anaconda3/envs/DL/lib/python3.9/site-packages/ipykernel_launcher.py", line 16, in <module>
      app.launch_new_instance()
    File "/home/zhkang/softwares/anaconda3/envs/DL/lib/python3.9/site-packages/traitlets/config/application.py", line 982, in launch_instance
      app.start()
    File "/home/zhkang/softwares/anaconda3/envs/DL/lib/python3.9/site-packages/ipykernel/kernelapp.py", line 677, in start
      self.io_loop.start()
    File "/home/zhkang/softwares/anaconda3/envs/DL/lib/python3.9/site-packages/tornado/platform/asyncio.py", line 215, in s